I see you've clicked onto this page. So you will see a glimpse of my coding style. It's far from done. There's a variables for me to understand exactly what the assignment was asking for the function object/ function to do. I would need to know what the user wanted to input. The condition statements can get messy. Should I assume there's only one condition (that's pretty easy, but I wanted it to be more complex)? I've been thinking about the problem and toying around with it. How do I build a function that doesn't have a long runtime (since the databases are going to be massively long) and where the user can type in a variety of specific information and get the table they were looking for? I'm being going back-and-forth changing/deleting things. I also want to make sure I can write it in a way that anyone can come in and see what each line means and why I wrote it. That is not going to be accomplishable for me within a couple of hours, especially because it's lots of information to swallow to get percision.

The data science part of finding the answer isn't too bad, but building a pipeline is going to take me longer. I was more focused on getting the pipeline built than the straight forward answer. 

In [3]:
import pandas as pd
import sqlite3
db = sqlite3.connect('claims.db')

In [2]:
# This class creates the table needed
class Table:
    
    def __init__(self,  select_columns, table, where_column, condition):
        self.table = table # string
        self.where_column = where_column # string
        self.condition = condition # list of strings, check if row has anything in [x,y]
        self.condition_str = ''
        self.select_columns = select_columns #columns for output, also list of strings
        self.conditions = ''
        self.d = []
        self.p = []
        
        #I'm using a loop to access all the P1-15 and D1-25 since they are indistinguishible 
        for i in range(1, 16):
            self.p.append('P' + str(i))
        for i in range(1, 26):
            self.d.append('D' + str(i))

    
    
    ### the long condition will be used if we are searching through D1-25 or P1-15
    def long_condition(self,p_d):
        for i in p_d:
            for j in self.condition:
                self.conditions += (i)+ ' LIKE (\'' + j + '%\')' + ' OR '
        self.conditions = self.conditions[:-4]
        return self.conditions
    

    def final_condition(self):
        if self.where_column[0]== 'P':
            self.conditions =  self.long_condition(self.p);
        elif self.where_column[0]=='D':
            self.conditions = self.long_condition(self.d);
        else:
            for i in self.condition:
                self.condition_str += i + ', '
            self.conditions =  (self.where_column + ' IN ' + '('+ self.condition_str[:-2] + ')')
        return self.conditions

    def new_table(self):
        self.final_condition()
        #now in order to use self.seearch_columns
        #I have to convert the datatype from list to str
        between_lst = ', '
        self.select_columns = between_lst.join(self.select_columns)
        return pd.read_sql_query('SELECT '  + self.select_columns 
                                 + ' FROM ' + self.table
                                 + ' WHERE ' + self.conditions + '',  db )
                    